In [65]:
from transformers import TFAutoModel

In [66]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [67]:
bert.summary()

Model: "tf_bert_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108310272 (413.17 MB)
Trainable params: 108310272 (413.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [68]:
import tensorflow as tf

In [69]:
#two cells
input_ids = tf.keras.layers.Input(shape=(512,),name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,),name='attention_mask', dtype='int32')

#transformers
embedding = bert.bert(input_ids,attention_mask=mask)[1] #max pooled activation

#classifiers head
x = tf.keras.layers.Dense(1024, activation='relu')(embedding)
y = tf.keras.layers.Dense(5, activation='softmax',name='outputs')(x)

In [70]:
model = tf.keras.Model(inputs=[input_ids,mask], outputs=y)

In [71]:
model.layers[2].trainable = False #Made Bert layer to be false

In [72]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 512)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 512)]                0         []                            
 )                                                                                                
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
                             ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, 512, 7                                     

In [73]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5,weight_decay=1e-6) #recommended value for bert
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [74]:
element_spec = ({'input_ids': tf.TensorSpec(shape=(16, 512), dtype=tf.int64, name=None),
  'attention_mask': tf.TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)},
 tf.TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [77]:
train_ds = tf.data.experimental.load('train', element_spec=element_spec)
val_ds = tf.data.experimental.load('val', element_spec=element_spec)


Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


In [78]:
history = model.fit(train_ds, validation_data=val_ds, epochs=3)

Epoch 1/3
  21/8778 [..............................] - ETA: 22:25:31 - loss: 1.2818 - accuracy: 0.5565

KeyboardInterrupt: 